In [ ]:
import pandas as pd
import operator
import time
import numpy as np

In [ ]:
config = {}

config["project_path"] = ".."
config["data_folder_path"] = "data"
config["temp_folder_path"] = "temp"

# input files
config['2018_2019_input'] = '2018_2019_data.xlsx'
config['nielsen_input'] = 'Nielsen 202001-05.xlsx'
config['performance_offtake'] = 'Performance Danone offtake 202001-05.xlsx'
config['spending_trade_support'] = 'spending trade support 202001-04.xlsx'
config['spending_CRM'] = 'spending CRM 202001-04.xlsx'
config['spending_media'] = 'spending media 2020Q1.xlsx'
config['spending_channel'] = 'Spending Channel AnP 202001-04.xlsx'
config['spending_PC'] = 'spending PC 202001-04.xlsx'

# output files
config['view_1_output_merg'] = 'Merge Nat Per&Inv.xlsx'
config['view_1_output_nav'] = 'National Unpivot 2.xlsx'

In [ ]:
def get_brand_name(SKU):
    if (SKU == 'AC') | (SKU == 'AP') | (SKU == 'AN'):
        return 'Aptamil'
    elif SKU == 'NC':
        return 'Nutrilon'
    return SKU

In [ ]:
offtake_df = pd.DataFrame(columns = ['Date', 'Brand', 'Province', 'Danone Offtake'])
ms_nutri = pd.DataFrame(columns = ['Date', 'Brand', 'Province', 'Market Share - Nutricia'])
ms_ap = pd.DataFrame(columns = ['Date', 'Brand', 'Province', 'Market Share - AP'])
ms_ac = pd.DataFrame(columns = ['Date', 'Brand', 'Province', 'Market Share - AC'])
ms_nc = pd.DataFrame(columns = ['Date', 'Brand', 'Province', 'Market Share - NC'])
wd_nutri = pd.DataFrame(columns = ['Date', 'Brand', 'Province', 'WTD - Nutricia'])
wd_ap = pd.DataFrame(columns = ['Date', 'Brand', 'Province', 'WTD - AP'])
wd_ac = pd.DataFrame(columns = ['Date', 'Brand', 'Province', 'WTD - AC'])
wd_nc = pd.DataFrame(columns = ['Date', 'Brand', 'Province', 'WTD - NC'])
ts = pd.DataFrame(columns = ['Date', 'Brand', 'Province', 'Trade Support'])
anp = pd.DataFrame(columns = ['Date', 'Brand', 'Province', 'Total A&P'])
chan = pd.DataFrame(columns = ['Date', 'Brand', 'Province', 'Total Channel'])
crm = pd.DataFrame(columns = ['Date', 'Brand', 'Province', 'Total CRM'])
media = pd.DataFrame(columns = ['Date', 'Brand', 'Province', 'Total Media'])
pc = pd.DataFrame(columns = ['Date', 'Brand', 'Province', 'Total PC'])

nav_columns = ['Date', 'Brand', 'Province', 'Attribute', 'Value']
view_1_output_nav = pd.DataFrame(columns = nav_columns)

## 2018 2019

In [ ]:
input_file_path = config['project_path'] + '/' \
                        + config['data_folder_path'] + '/' \
                        + config['2018_2019_input']

input_df = pd.read_excel(input_file_path, dtype=str)
input_df['Value'] = input_df['Value'].astype('float')
past_years_df = input_df

In [ ]:
def get_2018_2019(target_df, input_df, attribute_name):
    temp_df = input_df[input_df['Attribute'] == attribute_name].copy()
    temp_df = temp_df[['Date', 'Brand', 'Province', 'Value']]
    temp_df.columns = ['Date', 'Brand', 'Province', attribute_name]
    target_df = target_df.append(temp_df, ignore_index=True)
    return target_df

In [ ]:
all_columns = past_years_df['Attribute'].drop_duplicates().transpose().values.tolist()

In [ ]:
offtake_df = get_2018_2019(offtake_df, past_years_df, 'Danone Offtake')
ms_nutri = get_2018_2019(ms_nutri, past_years_df, 'Market Share - Nutricia')
ms_ap = get_2018_2019(ms_ap, past_years_df, 'Market Share - AP')
ms_ac = get_2018_2019(ms_ac, past_years_df, 'Market Share - AC')
ms_nc = get_2018_2019(ms_nc, past_years_df, 'Market Share - NC')
wd_nutri  =get_2018_2019(wd_nutri, past_years_df, 'WTD - Nutricia')
wd_ap = get_2018_2019(wd_ap, past_years_df, 'WTD - AP')
wd_ac = get_2018_2019(wd_ac, past_years_df, 'WTD - AC')
wd_nc = get_2018_2019(wd_nc, past_years_df, 'WTD - NC')
ts = get_2018_2019(ts, past_years_df, 'Trade Support')
anp = get_2018_2019(anp, past_years_df, 'Total A&P')
chan = get_2018_2019(chan, past_years_df, 'Total Channel')
crm = get_2018_2019(crm, past_years_df, 'Total CRM')
media = get_2018_2019(media, past_years_df, 'Total Media')
pc = get_2018_2019(pc, past_years_df, 'Total PC')

In [ ]:
view_1_output_nav = view_1_output_nav.append(past_years_df)

In [ ]:
def form_kpi_input(input_df):
    input_df = input_df[['Date', 'Brand', 'Province', 'KPI', 'KPI_value']]
    input_df.columns = ['Date', 'Brand', 'Province', 'Attribute', 'Value']
    return input_df

In [ ]:
def add_ly_column(input_df):
    ly_df =  input_df.copy()
    ly_df['Date'] = ly_df['Date'].astype(int)
    ly_df['Date'] = ly_df['Date'] + 100
    ly_df['Date'] = ly_df['Date'].astype(str)
    new_column_name = ly_df.columns[3] + ' LY'
    ly_df.columns = ['Date', 'Brand', 'Province', new_column_name]
    input_df = pd.merge(input_df, ly_df, on=['Date', 'Brand', 'Province'], how='left').fillna(0)
    
    return input_df

## Nielsen

In [ ]:
input_file_path = config['project_path'] + '/' \
                        + config['data_folder_path'] + '/' \
                        + config['nielsen_input']

input_df = pd.read_excel(input_file_path, dtype=str)
input_df['KPI_value'] = input_df['KPI_value'].astype('float')

In [ ]:
nutricia_df = input_df.loc[input_df['Brand'] == 'ELN']

temp_df = nutricia_df.loc[nutricia_df['KPI'] == 'Nielsen Value Market Share'][['Date', 'Brand', 'Province', 'KPI_value']]
temp_df.columns = ['Date', 'Brand', 'Province', 'Market Share - Nutricia']
temp_df['Brand'] = 'Aptamil'
ms_nutri = ms_nutri.append(temp_df)

temp_df = nutricia_df.loc[nutricia_df['KPI'] == 'WTD'][['Date', 'Brand', 'Province', 'KPI_value']]
temp_df.columns = ['Date', 'Brand', 'Province', 'WTD - Nutricia']
temp_df['Brand'] = 'Aptamil'
wd_nutri = wd_nutri.append(temp_df)

temp_df = nutricia_df.loc[nutricia_df['KPI'] == 'Nielsen Value Market Share']
temp_df = form_kpi_input(temp_df)
temp_df['Brand'] = 'Aptamil'
temp_df['Attribute'] = 'Market Share - Nutricia'
view_1_output_nav = view_1_output_nav.append(temp_df)

temp_df = nutricia_df.loc[nutricia_df['KPI'] == 'WTD']
temp_df = form_kpi_input(temp_df)
temp_df['Brand'] = 'Aptamil'
temp_df['Attribute'] = 'WTD - Nutrici'
view_1_output_nav = view_1_output_nav.append(temp_df)

In [ ]:
ms_temp_df = input_df.loc[(input_df['Brand'] != 'ELN') & (input_df['KPI'] == 'Nielsen Value Market Share')]

In [ ]:
def form_market_share_name(brand):
    if brand == 'AP':
        return 'Market Share - AP'
    elif brand == 'AC':
        return 'Market Share - AC'
    elif brand == 'NC':
        return 'Market Share - NC'
    return 'Market Share'

In [ ]:
ms_temp_df = form_kpi_input(ms_temp_df)
ms_temp_df['Attribute'] = ms_temp_df['Brand'].apply(lambda x : form_market_share_name(x))
ms_temp_df['Brand'] = ms_temp_df['Brand'].apply(lambda x : get_brand_name(x))
view_1_output_nav = view_1_output_nav.append(ms_temp_df)

In [ ]:
temp_df = ms_temp_df.loc[ms_temp_df['Attribute'] == 'Market Share - AP'][['Date', 'Brand', 'Province', 'Value']]
temp_df.columns = ['Date', 'Brand', 'Province', 'Market Share - AP']
ms_ap = ms_ap.append(temp_df)

temp_df = ms_temp_df.loc[ms_temp_df['Attribute'] == 'Market Share - AC'][['Date', 'Brand', 'Province', 'Value']]
temp_df.columns = ['Date', 'Brand', 'Province', 'Market Share - AC']
ms_ac = ms_ac.append(temp_df)

temp_df = ms_temp_df.loc[ms_temp_df['Attribute'] == 'Market Share - NC'][['Date', 'Brand', 'Province', 'Value']]
temp_df.columns = ['Date', 'Brand', 'Province', 'Market Share - NC']
ms_nc = ms_nc.append(temp_df)

In [ ]:
wtd_temp_df = input_df.loc[(input_df['Brand'] != 'ELN') & (input_df['KPI'] == 'WTD')]
wtd_temp_df = form_kpi_input(wtd_temp_df)
wtd_temp_df['Attribute'] = wtd_temp_df['Attribute'] + ' - ' + wtd_temp_df['Brand']
wtd_temp_df['Brand'] = wtd_temp_df['Brand'].apply(lambda x : get_brand_name(x))
view_1_output_nav = view_1_output_nav.append(wtd_temp_df)

In [ ]:
temp_df = wtd_temp_df.loc[wtd_temp_df['Attribute'] == 'WTD - AP'][['Date', 'Brand', 'Province', 'Value']]
temp_df.columns = ['Date', 'Brand', 'Province', 'WTD - AP']
wd_ap = wd_ap.append(temp_df)

temp_df = wtd_temp_df.loc[wtd_temp_df['Attribute'] == 'WTD - AC'][['Date', 'Brand', 'Province', 'Value']]
temp_df.columns = ['Date', 'Brand', 'Province', 'WTD - AC']
wd_ac = wd_ac.append(temp_df)

temp_df = wtd_temp_df.loc[wtd_temp_df['Attribute'] == 'WTD - NC'][['Date', 'Brand', 'Province', 'Value']]
temp_df.columns = ['Date', 'Brand', 'Province', 'WTD - NC']
wd_nc = wd_nc.append(temp_df)

## Danone Offtake

In [ ]:
input_file_path = config['project_path'] + '/' \
                        + config['temp_folder_path'] + '/' \
                        + config['performance_offtake']

input_df = pd.read_excel(input_file_path, dtype=str)
input_df['KPI_value'] = input_df['KPI_value'].astype('float')

In [ ]:
temp_df = input_df[['Date', 'Brand', 'Province', 'KPI_value']].copy()
temp_df.columns = ['Date', 'Brand', 'Province', 'Danone Offtake']
temp_df['Brand'] =  temp_df['Brand'].apply(lambda x : get_brand_name(x))
temp_df = temp_df.groupby(by = ['Date', 'Brand', 'Province'])['Danone Offtake'].sum().reset_index()
offtake_df = offtake_df.append(temp_df)

In [ ]:
temp_df = input_df.copy()
temp_df = form_kpi_input(temp_df)
temp_df['Brand'] = temp_df['Brand'].apply(lambda x : get_brand_name(x))
temp_df = temp_df.groupby(by = ['Date', 'Brand', 'Province', 'Attribute'])['Value'].sum().reset_index()
view_1_output_nav = view_1_output_nav.append(temp_df)

## Trade Support

In [ ]:
input_file_path = config['project_path'] + '/' \
                        + config['temp_folder_path'] + '/' \
                        + config['spending_trade_support']
input_df = pd.read_excel(input_file_path, dtype=str)
input_df['Value'] = input_df['Spending_value'].astype('float')

In [ ]:
temp_df = input_df[['Date', 'Brand', 'Province', 'Value']].copy()
temp_df.columns = ['Date', 'Brand', 'Province', 'Trade Support']
temp_df['Brand'] =  temp_df['Brand'].apply(lambda x : get_brand_name(x))
temp_df = temp_df.groupby(by = ['Date', 'Brand', 'Province'])['Trade Support'].sum().reset_index()
ts = ts.append(temp_df)

In [ ]:
temp_df = input_df[['Date', 'Brand', 'Province', 'Cost_type_1', 'Value']].copy()
temp_df.columns = nav_columns
temp_df['Brand'] = temp_df['Brand'].apply(lambda x : get_brand_name(x))
temp_df = temp_df.groupby(by = ['Date', 'Brand', 'Province', 'Attribute'])['Value'].sum().reset_index()
view_1_output_nav = view_1_output_nav.append(temp_df)

# Media

In [ ]:
input_file_path = config['project_path'] + '/' \
                        + config['temp_folder_path'] + '/' \
                        + config['spending_media']

input_df = pd.read_excel(input_file_path, dtype=str)
input_df['Value'] = input_df['Spending_value'].astype('float')
input_df['Brand'] = input_df['Brand'].apply(lambda x : get_brand_name(x))

In [ ]:
temp_df = input_df.groupby(by = ['Date', 'Brand', 'Province'])['Value'].sum().reset_index()
temp_df.columns = ['Date', 'Brand', 'Province', 'Total Media']
media = media.append(temp_df)

In [ ]:
temp_df = input_df.groupby(by = ['Date', 'Brand', 'Province'])['Value'].sum().reset_index()
temp_df['Attribute'] = 'Total Media'
temp_df = temp_df[['Date', 'Brand', 'Province', 'Attribute', 'Value']]
temp_df.columns = nav_columns
view_1_output_nav = view_1_output_nav.append(temp_df)

In [ ]:
temp_df = input_df.groupby(by = ['Date', 'Brand', 'Province', 'Cost_type_2'])['Value'].sum().reset_index()
temp_df['Attribute'] = temp_df['Cost_type_2']
temp_df = temp_df[['Date', 'Brand', 'Province', 'Attribute', 'Value']]
temp_df.columns = nav_columns
view_1_output_nav = view_1_output_nav.append(temp_df)

In [ ]:
temp_df = input_df.groupby(by = ['Date', 'Brand', 'Province', 'Cost_type_3'])['Value'].sum().reset_index()
temp_df['Attribute'] = temp_df['Cost_type_3']
temp_df = temp_df[['Date', 'Brand', 'Province', 'Attribute', 'Value']]
temp_df.columns = nav_columns
view_1_output_nav = view_1_output_nav.append(temp_df)

# CRM

In [ ]:
input_file_path = config['project_path'] + '/' \
                        + config['temp_folder_path'] + '/' \
                        + config['spending_CRM']

input_df = pd.read_excel(input_file_path, dtype=str)
input_df['Value'] = input_df['Spending_value'].astype('float')
input_df['Brand'] = input_df['Brand'].apply(lambda x : get_brand_name(x))

In [ ]:
temp_df = input_df.groupby(by = ['Date', 'Brand', 'Province'])['Value'].sum().reset_index()
temp_df.columns = ['Date', 'Brand', 'Province', 'Total CRM']
crm = crm.append(temp_df)

In [ ]:
temp_df = input_df.groupby(by = ['Date', 'Brand', 'Province'])['Value'].sum().reset_index()
temp_df['Attribute'] = 'Total CRM'
temp_df = temp_df[['Date', 'Brand', 'Province', 'Attribute', 'Value']]
temp_df.columns = nav_columns
view_1_output_nav = view_1_output_nav.append(temp_df)

In [ ]:
temp_df = input_df.groupby(by = ['Date', 'Brand', 'Province', 'Cost_type_2'])['Value'].sum().reset_index()
temp_df['Attribute'] = temp_df['Cost_type_2']
temp_df = temp_df[['Date', 'Brand', 'Province', 'Attribute', 'Value']]
temp_df.columns = nav_columns
view_1_output_nav = view_1_output_nav.append(temp_df)

# PC

In [ ]:
input_file_path = config['project_path'] + '/' \
                        + config['temp_folder_path'] + '/' \
                        + config['spending_PC']
input_df = pd.read_excel(input_file_path, dtype=str)
input_df['Value'] = input_df['Spending_value'].astype('float')
input_df['Brand'] = input_df['Brand'].apply(lambda x : get_brand_name(x))

In [ ]:
temp_df = input_df.groupby(by = ['Date', 'Brand', 'Province'])['Value'].sum().reset_index()
temp_df.columns = ['Date', 'Brand', 'Province', 'Total PC']
pc = pc.append(temp_df)

In [ ]:
temp_df = input_df.groupby(by = ['Date', 'Brand', 'Province'])['Value'].sum().reset_index()
temp_df['Attribute'] = 'Total PC'
temp_df = temp_df[['Date', 'Brand', 'Province', 'Attribute', 'Value']]
temp_df.columns = nav_columns
view_1_output_nav = view_1_output_nav.append(temp_df)

In [ ]:
temp_df = input_df.groupby(by = ['Date', 'Brand', 'Province', 'Cost_type_2'])['Value'].sum().reset_index()
temp_df['Attribute'] = temp_df['Cost_type_2']
temp_df = temp_df[['Date', 'Brand', 'Province', 'Attribute', 'Value']]
temp_df.columns = nav_columns
view_1_output_nav = view_1_output_nav.append(temp_df)

# Channel

In [ ]:
input_file_path = config['project_path'] + '/' \
                        + config['temp_folder_path'] + '/' \
                        + config['spending_channel']

input_df = pd.read_excel(input_file_path, dtype=str)

input_df['Value'] = input_df['Spending_value'].astype('float')
input_df['Brand'] = input_df['Brand'].apply(lambda x : get_brand_name(x))

In [ ]:
temp_df = input_df.groupby(by = ['Date', 'Brand', 'Province'])['Value'].sum().reset_index()
temp_df.columns = ['Date', 'Brand', 'Province', 'Total Channel']
chan = chan.append(temp_df)

In [ ]:
temp_df = input_df.groupby(by = ['Date', 'Brand', 'Province'])['Value'].sum().reset_index()
temp_df['Attribute'] = 'Total Channel'
temp_df = temp_df[['Date', 'Brand', 'Province', 'Attribute', 'Value']]
temp_df.columns = nav_columns
view_1_output_nav = view_1_output_nav.append(temp_df)

In [ ]:
temp_df = input_df.groupby(by = ['Date', 'Brand', 'Province', 'Cost_type_2'])['Value'].sum().reset_index()
temp_df['Attribute'] = temp_df['Cost_type_2']
temp_df = temp_df[['Date', 'Brand', 'Province', 'Attribute', 'Value']]
temp_df.columns = nav_columns
view_1_output_nav = view_1_output_nav.append(temp_df)

# Output result

In [ ]:
AnP_temp = view_1_output_nav.loc[(view_1_output_nav['Attribute'] == 'Total Media')
                                | (view_1_output_nav['Attribute'] == 'Total CRM')
                                | (view_1_output_nav['Attribute'] == 'Total PC')
                                | (view_1_output_nav['Attribute'] == 'Total Channel')]

In [ ]:
AnP_temp = AnP_temp.groupby(by = ['Date', 'Brand', 'Province'])['Value'].sum().reset_index()
AnP_temp['Attribute'] = 'Total A&P'
AnP_temp = AnP_temp[nav_columns]
view_1_output_nav = view_1_output_nav.append(AnP_temp)

In [ ]:
anp = AnP_temp[['Date', 'Brand', 'Province', 'Value']].copy()
anp.columns = ['Date', 'Brand', 'Province', 'Total A&P']

## nav view

In [ ]:
ly_df =  view_1_output_nav.copy()
ly_df['Date'] = ly_df['Date'].astype(int)
ly_df['Date'] = ly_df['Date'] + 100
ly_df['Date'] = ly_df['Date'].astype(str)
ly_df.columns = ['Date', 'Brand', 'Province', 'Attribute', 'Value LY']

In [ ]:
output_df = pd.merge(view_1_output_nav, ly_df, on=['Date', 'Brand', 'Province', 'Attribute'], how='left').fillna(0)

In [ ]:
an_df = output_df[~output_df['Attribute'].str.endswith('NC')][['Province', 'Attribute']].drop_duplicates()
nc_df = output_df[(~output_df['Attribute'].str.endswith('AC')) 
                  & ( ~output_df['Attribute'].str.endswith('AP'))] \
                    [['Province', 'Attribute']].drop_duplicates()
an_df['Brand'] = 'Aptamil'
nc_df['Brand'] = 'Nutrilon'
all_df = an_df.append(nc_df, ignore_index=True)

In [ ]:
all_dates_map = pd.DataFrame(columns=['Date', 'Brand', 'Province', 'Attribute'])
for i in range(1,13):
    year_month = "2018" + str(i).zfill(2)
    all_df['Date'] = year_month
    all_dates_map = all_dates_map.append(all_df, ignore_index=True)
for i in range(1,13):
    year_month = "2019" + str(i).zfill(2)
    all_df['Date'] = year_month
    all_dates_map = all_dates_map.append(all_df, ignore_index=True)
for i in range(1,13):
    year_month = "2020" + str(i).zfill(2)
    all_df['Date'] = year_month
    all_dates_map = all_dates_map.append(all_df, ignore_index=True)

In [ ]:
output_df = pd.merge(all_dates_map, output_df, on=['Date', 'Brand', 'Province', 'Attribute'], how='left')

output_df = output_df.fillna({'Value': 0, 'Value LY': 0})

In [ ]:
def format_yearmonth(year_month):
    return time.strftime('%Y/%m/%d', time.strptime(year_month,'%Y%m'))

output_df['Date'] = output_df['Date'].apply(lambda x : format_yearmonth(x))

In [ ]:
output_file_path = config['project_path'] + '/' \
                        + config['temp_folder_path'] + '/' \
                        + config['view_1_output_nav'] 

output_df.to_excel(output_file_path, index=False)

## merg view

In [ ]:
offtake_df = add_ly_column(offtake_df)
ms_nutri = add_ly_column(ms_nutri)
ms_ap = add_ly_column(ms_ap)
ms_ac = add_ly_column(ms_ac)
ms_nc = add_ly_column(ms_nc)
wd_nutri = add_ly_column(wd_nutri)
wd_ap = add_ly_column(wd_ap)
wd_ac = add_ly_column(wd_ac)
wd_nc = add_ly_column(wd_nc)
ts = add_ly_column(ts)
anp = add_ly_column(anp)
chan = add_ly_column(chan)
crm = add_ly_column(crm)
media = add_ly_column(media)
pc = add_ly_column(pc)

In [ ]:
output_df = offtake_df
output_df = pd.merge(output_df, ms_nutri, on=['Date', 'Brand', 'Province'], how='outer')
output_df = pd.merge(output_df, ms_ap, on=['Date', 'Brand', 'Province'], how='outer')
output_df = pd.merge(output_df, ms_ac, on=['Date', 'Brand', 'Province'], how='outer')
output_df = pd.merge(output_df, ms_nc, on=['Date', 'Brand', 'Province'], how='outer')
output_df = pd.merge(output_df, wd_nutri, on=['Date', 'Brand', 'Province'], how='outer')
output_df = pd.merge(output_df, wd_ap, on=['Date', 'Brand', 'Province'], how='outer')
output_df = pd.merge(output_df, wd_ac, on=['Date', 'Brand', 'Province'], how='outer')
output_df = pd.merge(output_df, wd_nc, on=['Date', 'Brand', 'Province'], how='outer')
output_df = pd.merge(output_df, ts, on=['Date', 'Brand', 'Province'], how='outer')
output_df = pd.merge(output_df, anp, on=['Date', 'Brand', 'Province'], how='outer')
output_df = pd.merge(output_df, chan, on=['Date', 'Brand', 'Province'], how='outer')
output_df = pd.merge(output_df, crm, on=['Date', 'Brand', 'Province'], how='outer')
output_df = pd.merge(output_df, media, on=['Date', 'Brand', 'Province'], how='outer')
output_df = pd.merge(output_df, pc, on=['Date', 'Brand', 'Province'], how='outer')

In [ ]:
output_df = output_df.fillna(0)
output_df['Date'] = output_df['Date'].apply(lambda x : format_yearmonth(x))

In [ ]:
output_file_path = config['project_path'] + '/' \
                        + config['temp_folder_path'] + '/' \
                        + config['view_1_output_merg'] 

output_df.to_excel(output_file_path, index=False)